In [1]:
!gcloud config set project  bwh-midrc-rapid-res-1655321320

Updated property [core/project].


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
  project_name = "bwh-midrc-rapid-res-1655321320"
  bucket_name = "midrc-analysis-bwh"
  bucket_path = "bpr-results/"

In [4]:
import os
import shutil
import glob
import json
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.colors import ListedColormap
import numpy as np

In [5]:
!pip install dicomweb-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.1 MB/s 
     |████████████████████████████████| 3.1 MB 10.9 MB/s 
     |████████████████████████████████| 2.0 MB 48.6 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=4fc4d7452b87e8e5e257a22b6b64f50afe7d207b77fed04be9317b18fd95089f
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [25]:
#install nrrd
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path, delimiter='/')
for blob in blobs:
    bn = blob.name
    bs = bn.split('.')
    if bs[-1] == 'json':
      bss = bn.split('/')[1]
      bs3 = bss.split('.')
      bs4 = '.'.join(bs3[:-1])
      series_instance_uids.append(bs4)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [ ]:
print(len(series_instance_uids))

495


In [ ]:
path_downloaded = '/content/downloaded_data'
path_nifti = '/content/nifti_data'
path_json =  '/content/json_data'
path_nrrd = '/content/nrrd_data'

for x in [path_downloaded, path_nifti, path_json, path_nrrd]:
  if os.path.isdir(x):
    try:
      shutil.rmtree(x)
    except OSError as err:
      print("Error: %s : %s" % (x, err.strerror))  
  os.mkdir(x)


In [ ]:
fn_seg_nrrd = os.path.join(path_nrrd, 'seg_%s.nrrd' % seriesuid)
nrrd.write( fn_seg_nrrd, s,  header = ctheader)

In [ ]:
cmd1 = "gsutil cp %s gs://%s/%s" % (fn_seg_nrrd, bucket_name, bucket_path)
print(cmd1)
os.system(cmd1)

cmd2 = "gsutil cp %s gs://%s/%s" % (fn_ct_nrrd, bucket_name, bucket_path)
print(cmd2)
os.system(cmd2)

gsutil cp /content/nrrd_data/seg_1.3.6.1.4.1.5962.99.1.3492129840.282517718.1618399864692.46885.0.nrrd gs://midrc-analysis-bwh/bpr-results/
gsutil cp /content/nrrd_data/ct_1.3.6.1.4.1.5962.99.1.3492129840.282517718.1618399864692.46885.0.nrrd gs://midrc-analysis-bwh/bpr-results/


0

In [ ]:
# Define slices with segmented lungs

l1_list = list(np.array(np.where((np.sum((s==1),axis=(0,1))>0)==True)[0]).astype(int))
l1_min = min(l1_list)
l1_max = max(l1_list)
#print(l1_min, l1_max)

l2_list = list(np.array(np.where((np.sum((s==2),axis=(0,1))>0)==True)[0]).astype(int))
l2_min = min(l2_list)
l2_max = max(l2_list)
#print(l2_min, l2_max)


lmin_all = min([l1_min, l1_max,l2_min, l2_max])
lmax_all = max([l1_min, l1_max,l2_min, l2_max])
llist_out = [lmin_all, lmax_all]
lungs = list(range(lmin_all, lmax_all+1, 1))

print(llist_out)
print(lungs)

[155, 177]
[155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177]


In [ ]:
# Use BPR json for slice classification  

file_json = '%s.json' % seriesuid
cmd = "gsutil cp gs://%s/%s%s  %s" % (bucket_name, bucket_path, file_json, path_json)
print(cmd)
os.system(cmd)

fileObject = open("%s/%s" % (path_json, file_json), "r")
jsonContent = fileObject.read()
aList = json.loads(jsonContent)

for (bodypart, zlist) in aList['body part examined'].items():
    print(bodypart, zlist)

In [ ]:
for seriesuid in series_instance_uids:
  

  file_json = '%s.json' % seriesuid
  cmd = "gsutil cp gs://%s/%s%s  %s" % (bucket_name, bucket_path, file_json, path_json)
  print(cmd)
  os.system(cmd)

In [ ]:
k = 0

for seriesuid in series_instance_uids:

  file_json = seriesuid + '.json'

  fileObject = open("%s/%s" % (path_json, file_json), "r")
  jsonContent = fileObject.read()
  aList = json.loads(jsonContent)

  bp = aList['body part examined']
  ch = bp['chest']
  l = len(ch)
  if l > 0:
    k=k+1

print(k)

479


In [ ]:
file_gs_uri = "gs://%s/%s/%s.json" % (bucket_name, bucket_path, seriesuid)

json_list = glob(os.path.join( pat_dir_path_json,'*.json'))
  print(len(json_list), 'files for this patient')

  for jn in json_list:
    fileObject = open(jn, "r")
    jsonContent = fileObject.read()
    aList = json.loads(jsonContent)

    #print(aList['body part examined'])

    jj = os.path.split(jn)
    jjp = jj[0] # path
    jjn = jj[1] # json file name
    bpfn = os.path.join(jjp, jjn.split('.')[0]+'_bodypart_regr.txt')

    f = open(bpfn,'w')
    for (bodypart, zlist) in aList['body part examined'].items():
      #print(jn)
      print(bodypart, zlist)
      if (len(zlist)>0):
        f.write(bodypart+'\n')
    f.close()


In [ ]:
# save body part to a single .csv
fn_out = '/content/bpr_by_series.csv'
print(bucket_name, bucket_path)

df = pd.DataFrame(columns = ['SeriesUID','bodypart'])
k=0
storage_client = storage.Client()
blobs = storage_client.list_blobs(bucket_name)
for b in blobs:
  #print(b.name)
  p = os.path.split(b.name)
  # skip non-axial and folders
  if p[0]==bucket_path and p[1].split('.')[-1]=='json':
    k +=1
    seriesuid = '.'.join(p[1].split('.')[:-1])
    #print(b.name)
    contents = storage_client.bucket(bucket_name).blob(b.name).download_as_string()
    jdata = json.loads(contents)
    for (bodypart, zlist) in jdata['body part examined'].items():
      if len(zlist) >0:
        df = df.append( {'SeriesUID': seriesuid, 'bodypart': bodypart }, ignore_index = True)
      if k%10==0:
        print(k,' ', end='')
      if k%120==0:
        print('')
print('\nfiles processed', k)
print(df)
df.to_csv(fn_out, index=False)
cmd = 'gsutil cp %s gs://%s/%s/results-summary/' % (fn_out, bucket_name,bucket_path)
print(cmd)
os.system(cmd)

In [ ]:
# cleanup for the next iteration
  
for x in [path_downloaded, path_nifti, path_nrrd, path_json]:
  if os.path.exists(x):
    try:
      shutil.rmtree(x)
    except OSError as err:
      print("Error: %s : %s" % (x, err.strerror))